In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

import pickle

all_texts = pickle.load(open('/content/drive/My Drive/psych/klamanie/dane/all_texts.pkl', 'rb'))
all_labels = pickle.load(open('/content/drive/My Drive/psych/klamanie/dane/all_labels.pkl', 'rb'))

drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
project_id = 'klamanie'
!gcloud config set project klamanie

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [ ]:
!gsutil ls

gs://klamanie-trening/


In [ ]:
!gsutil ls gs://klamanie-trening/

gs://klamanie-trening/pisemne/
gs://klamanie-trening/temp/
gs://klamanie-trening/transkrypcje/



### ------------------------------------ BERT Large

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade mlxtend

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 68.7 MB/s 
     |████████████████████████████████| 61 kB 475 kB/s 
     |████████████████████████████████| 596 kB 76.9 MB/s 
     |████████████████████████████████| 3.3 MB 76.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.3 MB 5.4 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
import tensorflow as tf

import os
print(tf.__version__)

2.7.0


In [ ]:
from transformers import (
    TF2_WEIGHTS_NAME,
    BertConfig,
    BertTokenizer,
    AutoTokenizer,
    TFBertForSequenceClassification,
    AutoTokenizer,
    create_optimizer)

def get_BERT_model(model_name, lower_case=False):

  MODEL_CLASSES = {"bert": (BertConfig, TFBertForSequenceClassification, AutoTokenizer)}
  config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

  config = config_class.from_pretrained(model_name, num_labels=2)
  model = model_class.from_pretrained(
                  model_name,
                  from_pt=True,
                  config=config)

  tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=lower_case)

  model.layers[-1].activation = tf.keras.activations.sigmoid
  return model, tokenizer

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report
from transformers.optimization_tf import AdamWeightDecay
from transformers import TFTrainer, TFTrainingArguments
import transformers
import gc

transformers.logging.set_verbosity_error()

all_results = {}

for text_type in ['p', 't']:

  kf = KFold(n_splits=10, shuffle=True, random_state=42)
  fold_nr=0
  
  texts = np.array(all_texts[text_type])
  labels = np.array(all_labels[text_type])

  all_results[text_type] = []

  for train_index, test_index in kf.split(texts):

    print(f'Fold_nr= {fold_nr} on type {text_type}')  

    X_train_val, X_test = texts[train_index], texts[test_index]
    y_train_val, y_test = labels[train_index], labels[test_index]


    #############################################

    #BERT_MODEL="allegro/herbert-base-cased", "allegro/herbert-large-cased", "dkleczek/bert-base-polish-uncased-v1","bert-base-uncased"    
    for BERT_MODEL in [ 
                       {'name':"allegro/herbert-large-cased", 'lowcase':False, 'batch':4}
                       ]:

      print(f"   training {BERT_MODEL['name']}")
      
      # split train_val into train and validation:
      train_texts, val_texts, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.05, random_state=42)
      test_texts = X_test

      training_args = TFTrainingArguments(
          output_dir='gs://klamanie-trening/temp/results-'+BERT_MODEL['name'].replace('/','-')+'-'+str(fold_nr),          # output directory
          overwrite_output_dir=True,
          eval_accumulation_steps=5,
          gradient_accumulation_steps=4,
          tpu_num_cores=8,
          num_train_epochs=10,              # total number of training epochs
          per_device_train_batch_size=BERT_MODEL['batch'],  # batch size per device during training
          per_device_eval_batch_size=BERT_MODEL['batch'],   # batch size for evaluation
          warmup_steps=500,                # number of warmup steps for learning rate scheduler
          weight_decay=0.01,               # strength of weight decay
          logging_dir='gs://klamanie-trening/temp/logs',            # directory for storing logs
          logging_steps=10,
          evaluation_strategy="steps",     # Evaluation is done at the end of each epoch, alternatives: "steps", "epoch"
          save_strategy="steps",           # Save is done at the end of each epoch, alternatives: "steps", "epoch"
          # save_steps=500                 # Number of updates steps before two checkpoint saves if save_strategy="steps"
          eval_steps = 1,
          save_total_limit = 5,
          load_best_model_at_end=True,     # Whether or not to load the best model found during training at the end of training.
      )

      with training_args.strategy.scope():
          model, tokenizer = get_BERT_model(BERT_MODEL['name'], lower_case=BERT_MODEL['lowcase'])

          train_encodings = tokenizer(train_texts.tolist() , padding=True, truncation=True)
          val_encodings = tokenizer(val_texts.tolist() , padding=True, truncation=True)
          test_encodings = tokenizer(test_texts.tolist() ,padding=True, truncation=True)

          train_dataset = tf.data.Dataset.from_tensor_slices(( dict(train_encodings), y_train ))
          val_dataset = tf.data.Dataset.from_tensor_slices(( dict(val_encodings), y_val ))
          test_dataset = tf.data.Dataset.from_tensor_slices(( dict(test_encodings), y_test ))

          trainer = TFTrainer(
              model=model,                         # the instantiated 🤗 Transformers model to be trained
              args=training_args,                  # training arguments, defined above
              train_dataset=train_dataset,         # training dataset
              eval_dataset=val_dataset             # evaluation dataset
          )

          trainer.train()
          predictions = trainer.predict(test_dataset)

      y_pred = np.argmax(predictions.predictions, axis=1)
      report = classification_report(y_pred, y_test, output_dict=True)
      print(report)
      model_results = { 'report':report, 'y_pred':y_pred, 'y_true':y_test , 'fold_nr':fold_nr, 'model':BERT_MODEL['name'], 'text_type':text_type }
      all_results[text_type].append(model_results)
      fname = BERT_MODEL['name'].replace('/','-')+'-'+str(fold_nr)+'.pkl'

      del model
      tf.keras.backend.clear_session()
      gc.collect()

    fold_nr += 1


from sklearn.metrics import classification_report
# http://rasbt.github.io/mlxtend/user_guide/evaluate/cochrans_q/
from mlxtend.evaluate import cochrans_q
import numpy as np

#model_results = { 'report':report, 'y_pred':y_pred, 'y_true':y_test , 'fold_nr':fold_nr, 'model':BERT_MODEL['name'], 'text_type':text_type }
#all_results[text_type].append(model_results)


for text_type in ['p', 't']:
  print(f"text type: {text_type}")  
  y_pred = {}
  y_true = {}

  for result_dict in all_results[text_type]:
    if result_dict['model'] not in y_pred:
      y_pred[ result_dict['model'] ] = []
      y_true[ result_dict['model'] ] = []
    y_pred[ result_dict['model'] ].extend( result_dict['y_pred'].astype(int) )
    y_true[ result_dict['model'] ].extend( result_dict['y_true'].astype(int) )

  for model in y_pred.keys():
    print(model)
    print(classification_report(y_true[model], y_pred[model]))

  q, p_value = cochrans_q(np.array(y_true["allegro/herbert-large-cased"]))

  print('Q: %.3f' % q)
  print('p-value: %.3f' % p_value)


Fold_nr= 0 on type p
   training allegro/herbert-large-cased
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.38.29.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.29.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
/usr/local/lib/python3.7/dist-packages/transformers/trainer_tf.py:114: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/master/examples/tensorflow
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const_1:0' shape=() dtype=int32>, <tf.Ten

{'0': {'precision': 0.46153846153846156, 'recall': 0.6206896551724138, 'f1-score': 0.5294117647058824, 'support': 29}, '1': {'precision': 0.6764705882352942, 'recall': 0.5227272727272727, 'f1-score': 0.5897435897435898, 'support': 44}, 'accuracy': 0.5616438356164384, 'macro avg': {'precision': 0.5690045248868778, 'recall': 0.5717084639498433, 'f1-score': 0.5595776772247361, 'support': 73}, 'weighted avg': {'precision': 0.5910865926981963, 'recall': 0.5616438356164384, 'f1-score': 0.5657761523998429, 'support': 73}}
text type: p
allegro/herbert-large-cased
              precision    recall  f1-score   support

           0       0.59      0.62      0.60       378
           1       0.60      0.57      0.58       379

    accuracy                           0.59       757
   macro avg       0.59      0.59      0.59       757
weighted avg       0.59      0.59      0.59       757



ValueError: ignored